In [1]:
import copy

<span style="color:red">Start Position</span>
<br>
<span style="color:blue">Current Position</span>
<br>
<span style="color:green">Target Position</span>
<br>


    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    UNDERLINE = '\033[4m'
    RESET = '\033[0m'

In [2]:
# def solve_dfs(self, moves=[], path=[]):
# #     print(len(moves), moves)
#     pos = copy.deepcopy(self.pos)
#     state = copy.deepcopy(self.state)
#     if moves==[]:
#         self.positions = []
#         self.store = []
#     #    
# #     print(self.pos, self.pos not in self.positions)
#     if self.is_at_target():
#         self.store.append(moves)
#     elif self.pos in self.positions:
        
#     else:
#         self.positions.append(self.pos)
#         for m in self.moves:
#             try:
#                 m()
# #                 print(m.__name__)
#                 solutions = solve_dfs(self, moves+[m.__name__])
#                 self.store.extend(solutions)
#             except AssertionError:
#                 pass
#             self.pos = pos
#             self.state = state
#     # #
# #     print(self.positions)
#     return store

In [68]:
def solve_dfs(self, moves=[], path=[], best=None, thresh=0):
    """
    moves: contains history of moves executed previosuly.
    path: contains history of positions visited previously.
    best: Minimium distance between cursor postion and target position in the current session, 
            used to keep track of and go in correct direction.
    thresh: used for strict solving, less values means tighter paths with less steps. 
            If the game needs special moves try increasing thresh.
    """
#     print(len(moves), moves)#, path)
    pos = copy.deepcopy(self.pos)
    state = copy.deepcopy(self.state)
    if moves==[]:
        self.positions = []
        self.store = []
    if best is None:
        best = self.distance_to_target()
    if self.is_at_target():
        # Check If we are already at target we would now terminate the recursion tree and return the solution.
        self.store.append((moves, path))
    elif self.pos in self.positions:
        # Check If the position has already been visited in a successfull move 
        # we would now terminate the recursion tree and return the solution.
        for m, p in self.store:
            try:
                idx = p.index(self.pos, -1)
                self.store.append(moves+[m[idx+1:], path+p[idx+1:]])
            except ValueError:
                pass
    else:        
        # This position is new and is not target position  
        # we would now explore in all directions of the position untill the threshold allows.
        self.positions.append(self.pos)
        for m in self.moves:
            try:
                m()
                best = min(best, self.distance_to_target())
                if self.distance_to_target() <= best+thresh:
                    solve_dfs(self, moves+[m.__name__], path+[self.pos], best)
            except AssertionError:
                pass
            self.pos = pos
            self.state = state
    return self.store

In [69]:
#
def sort_helper(x):
    return len(x[0])

In [70]:
class Cursor():
    map = None
    """
    Pos: Array coordinates 
    State: standing, lying horizontally, and lying vertically
           st, lh and lv
    """
    def __init__(self, pos, map):
        self.init_pos = self.pos = pos
        self.state = self.check_state()
        self.map = map
    
    def check_state(self, pos=None):
        if pos is None:
            pos = self.pos
        if type(pos[0]) is list:
            return 'lv'
        elif type(pos[1]) is list:
            return 'lh'
        else:
            return 'st'
    
    def distance_to_target(self, pos=None):
        if pos is None:
            pos = self.pos
        if type(pos[0]) is list:
            a = self.map.target_pos[0]-(sum(pos[0])*.5)
            b = self.map.target_pos[1]-pos[1]
        elif type(pos[1]) is list:
            a = self.map.target_pos[0]-pos[0]
            b = self.map.target_pos[1]-(sum(pos[1])*.5)
        else:
            a = self.map.target_pos[0]-pos[0]
            b = self.map.target_pos[1]-pos[1]
        return (a**2+b**2)**.5 
        
    def backup_state(self):
        self._state = self.state
        self._pos = self.pos
    
    def restore_state(self):
        self.state = self._state
        self.pos = self._pos
            
    def apply_state(self, pos, state):
#         print(self.pos, self.state)
#         print(pos, state)
        self.backup_state()
        self.assert_valid_pos(pos, state)
        self.pos = pos
        self.state = state
        
    def calculate_state(self, direction):
        p = self.pos
        if self.state == 'st':
            if direction == 'right':
                state = 'lh'
                pos = [p[0], [p[1]+1, p[1]+2]]
            elif direction == 'left':
                state = 'lh'
                pos = [p[0], [p[1]-2, p[1]-1]]            
            elif direction == 'up':
                state = 'lv'
                pos = [[p[0]-2, p[0]-1], p[1]]  
            elif direction == 'down':
                state = 'lv'
                pos = [[p[0]+1, p[0]+2], p[1]]                
        elif self.state == 'lh':
            if direction == 'right':
                state = 'st'
                pos = [p[0], p[1][1]+1]
            elif direction == 'left':
                state = 'st'
                pos = [p[0], p[1][0]-1]
            elif direction == 'up':
                state = 'lh'
                pos = [p[0]-1, p[1]]
            elif direction == 'down':
                state = 'lh'
                pos = [p[0]+1, p[1]]            
        else: # lv
            if direction == 'right':
                state = 'lv'
                pos = [p[0], p[1]+1]
            elif direction == 'left':
                state = 'lv'
                pos = [p[0], p[1]-1]            
            elif direction == 'up':
                state = 'st'
                pos = [p[0][0]-1, p[1]]     
            elif direction == 'down':
                state = 'st'
                pos = [p[0][1]+1, p[1]]     
        return pos, state
    
    def right(self):
        pos, state = self.calculate_state('right')
        self.apply_state(pos, state)        
    
    def left(self):
        pos, state = self.calculate_state('left')
        self.apply_state(pos, state)      
    
    def up(self):
        pos, state = self.calculate_state('up')
        self.apply_state(pos, state) 
    
    def down(self):
        pos, state = self.calculate_state('down')
        self.apply_state(pos, state)      
    
    def assert_valid_pos(self, pos=None, state=None):
        if not self.map.allow_invalid_pos:
            assert self.is_valid_pos(pos)==True
    
    def is_valid_pos(self, pos=None):
#         print(pos)
        if pos is None:
            pos = self.pos
        if type(pos[0]) is list:
            if min(pos[0]) < 0 or max(pos[0]) >= self.map.rows:
                return False
            if pos[1] < 0 or pos[1] >= self.map.cols:
                return False
            return self.map.grid[pos[0][0]][pos[1]] and self.map.grid[pos[0][1]][pos[1]]
        elif type(pos[1]) is list:
            if pos[0] < 0 or pos[0] >= self.map.rows:
                return False
            if min(pos[1]) < 0 or max(pos[1]) >= self.map.cols:
                return False
            return self.map.grid[pos[0]][pos[1][0]] and self.map.grid[pos[0]][pos[1][1]]
        if pos[0] < 0 or pos[0] >= self.map.rows or pos[1] < 0 or pos[1] >= self.map.cols:
            return False
        return self.map.grid[pos[0]][pos[1]]
    
    def is_at_start(self):
        return self.pos == self.map.start_pos
    
    def is_at_target(self):
        return self.pos == self.map.target_pos
    
    @property
    def moves(self):
        return [
            self.up,
            self.down,
            self.left,
            self.right
        ]
    
    def solve_dfs(self, max_thresh=5):
        for i in range(max_thresh+1):
            solutions = solve_dfs(self)
            if len(solutions)>0:
                solutions.sort(key=sort_helper)
                return solutions

    def get_best_solution(self, algos=['dfs'], plot=False):
        solutions = []
        for a in algos:
            solutions.extend(getattr(self, 'solve_'+a)())
        #
        if len(solutions)>0:
            solutions.sort(key=sort_helper)
            if plot:
                self.map.plot(path=solutions[0][0])
            else:
                return solutions[0]
        

In [71]:
class Map():
    grid = None
    terminal = None
    schema = None
    cursor = None
    allow_invalid_pos = False
    def __init__(self, schema:'list[list]', start_pos=None, target_pos=None):
        self.schema = schema
        self.start_pos = start_pos
        self.target_pos = target_pos
        self.init_grid()
            
    def init_cursor(self, current_pos=None):
        if current_pos is None:
            current_pos = self.start_pos
        self.cursor = Cursor(current_pos, self)
    
    def init_grid(self):
        self.rows = len(schema)
        self.cols = 0
        for r in self.schema:
            for entry in r:
                self.cols = max(self.cols, entry[1])
        self.cols+=1
        self.grid = []
        self.terminal = []
        for r in self.schema:
            row = [False]*self.cols
            row2 = [False]*self.cols
            for entry  in r:
                for i in range(entry[0]-1, entry[1]):
                    row2[i]=False
                row2[entry[1]]=True
                #
                for i in range(entry[0], entry[1]+1):
                    row[i]=True
            self.grid.append(row)
            self.terminal.append(row2)
        
    def plot(self, path=None, header=True):
        if path is None and header:
            print(f'-- Map Plot {self.rows}X{self.cols}')
        elif path is not None :
            print(f'-- Map Plot {self.rows}X{self.cols} With Path {len(path)} Steps: ', "-".join(path))
            pos = copy.deepcopy(self.cursor.pos)
            state = copy.deepcopy(self.cursor.state)
            ex = None
            try:
                print(f"\nHome | State: {self.cursor.pos} {self.cursor.state}")
                self.plot(header=False)
                for i, m in enumerate(path):
                    getattr(self.cursor, m)()
                    print(f"\nStep {i}: {m} | State: {self.cursor.pos} {self.cursor.state}")
                    self.plot(header=False)
            except Exception as e:
                ex = e
            finally:
                self.cursor.pos = pos
                self.cursor.state = state
                if ex is not None:
                    raise ex
                return
        print('\n    ', end='')
        for i in range(self.cols):
            print(i, end=' ')
        print(' ')
        for r in range(self.rows):
            print(('   '+str(r))[-3:], end=' ')
            # Top
            for c in range(self.cols):
                if self.grid[r][c]:
                    col = '+ '
                    if self.terminal[r][c]:
                        col+='+ '
                elif r>0 and self.grid[r-1][c]:
                    col = '+ '
                else:
                    col = '  '
                print(col, end='')
            print('')
            print('   ', end=' ')
            # Middle|
            for c in range(self.cols):
                # Infer
                if self.grid[r][c]:
                    col = '|'
                else:
                    col = ' '
                empty = True
                if self.cursor is not None:
                    pos = self.cursor.pos
                    if pos == [r,c]: # st
                        col+='\033[34mX\033[0m'
                        empty= False
                    elif self.cursor.state == 'lv':
                        if ([pos[0][0], pos[1]]==[r,c]) or ([pos[0][1], pos[1]]==[r,c]):
                            col+='\033[34mX\033[0m'
                            empty= False                    
                    elif self.cursor.state == 'lh':
                        if ([pos[0], pos[1][0]]==[r,c]) or ([pos[0], pos[1][1]]==[r,c]): # lh
                            col+='\033[34mX\033[0m'
                            empty= False
                #
                if empty:
                    if self.start_pos == [r, c]:
                        col+='\033[31mX\033[0m'
                        empty= False
                    elif self.target_pos == [r, c]:
                        col+='\033[32mX\033[0m'
                        empty= False
                if empty:
                    if self.grid[r][c]:
                        col+='\033[33m-\033[0m'
                    else:
                        col+=' '
                if self.terminal[r][c]:
                    col+='| '
                print(col, end='')
            # End
            if r==self.rows-1:
                print('')
                print('   ', end=' ')
                for c in range(self.cols):
                    if self.grid[r][c]:
                        col = '+ '
                    else:
                        col = '  '
                    if self.terminal[r][c]:
                        col+='+ '
                    print(col, end='')
            print('')

In [72]:
schema = [
    [(0,9)],
    [(0,2), (2, 4)],
    [(1, 3), (4, 8)]
]

In [73]:
schema = [
    [(0,2)],
    [(0,5)],
    [(0,9)],
    [(1,9)],
    [(5,9)],
    [(6,8)]
]

In [74]:
m = Map(schema, [1,1], [4,7])
m.init_cursor()

In [75]:
m.grid

[[True, True, True, False, False, False, False, False, False, False],
 [True, True, True, True, True, True, False, False, False, False],
 [True, True, True, True, True, True, True, True, True, True],
 [False, True, True, True, True, True, True, True, True, True],
 [False, False, False, False, False, True, True, True, True, True],
 [False, False, False, False, False, False, True, True, True, False]]

In [76]:
m.terminal

[[False, False, True, False, False, False, False, False, False, False],
 [False, False, False, False, False, True, False, False, False, False],
 [False, False, False, False, False, False, False, False, False, True],
 [False, False, False, False, False, False, False, False, False, True],
 [False, False, False, False, False, False, False, False, False, True],
 [False, False, False, False, False, False, False, False, True, False]]

In [77]:
m.plot()

-- Map Plot 6X10

    0 1 2 3 4 5 6 7 8 9  
  0 + + + +               
    |-|-|-|               
  1 + + + + + + +         
    |-|X|-|-|-|-|         
  2 + + + + + + + + + + + 
    |-|-|-|-|-|-|-|-|-|-| 
  3 + + + + + + + + + + + 
      |-|-|-|-|-|-|-|-|-| 
  4   + + + + + + + + + + 
              |-|-|X|-|-| 
  5           + + + + + + 
                |-|-|-|   
                + + + +   


In [78]:
cur = m.cursor

In [79]:
cur.is_at_start()

True

In [80]:
cur.is_at_target()

False

In [81]:
cur.get_best_solution(plot=True)

-- Map Plot 6X10 With Path 7 Steps:  right-down-down-right-right-down-right

Home | State: [1, 1] st

    0 1 2 3 4 5 6 7 8 9  
  0 + + + +               
    |-|-|-|               
  1 + + + + + + +         
    |-|X|-|-|-|-|         
  2 + + + + + + + + + + + 
    |-|-|-|-|-|-|-|-|-|-| 
  3 + + + + + + + + + + + 
      |-|-|-|-|-|-|-|-|-| 
  4   + + + + + + + + + + 
              |-|-|X|-|-| 
  5           + + + + + + 
                |-|-|-|   
                + + + +   

Step 0: right | State: [1, [2, 3]] lh

    0 1 2 3 4 5 6 7 8 9  
  0 + + + +               
    |-|-|-|               
  1 + + + + + + +         
    |-|X|X|X|-|-|         
  2 + + + + + + + + + + + 
    |-|-|-|-|-|-|-|-|-|-| 
  3 + + + + + + + + + + + 
      |-|-|-|-|-|-|-|-|-| 
  4   + + + + + + + + + + 
              |-|-|X|-|-| 
  5           + + + + + + 
                |-|-|-|   
                + + + +   

Step 1: down | State: [2, [2, 3]] lh

    0 1 2 3 4 5 6 7 8 9  
  0 + + + +               
    |-|-|-| 